In [56]:
import pandas as pd
import math
import requests
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import re
import csv
import time

In [57]:
df = pd.read_csv('Data/madlan_data.csv')
# df = df.dropna(subset =['Baths'])
print(df.shape)

def fix_BuildingClass(df):
    df.rename(columns = {'BuildingClass':'Asset_type'}, inplace = True)
    df = df[df['Asset_type'].apply(lambda x:x in ['flat','gardenapartment','roofflat','building','studio'])]
    return df

df = fix_BuildingClass(df)
print(df.shape)

(428, 18)
(354, 18)


In [58]:
def get_gush_chelka_api(address):
    headers = {
        'sec-ch-ua': '"Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"',
        'auth_data': '{"api_token":"03387185-da2d-4cb4-a3c6-2292a3df7915","user_token":"","domain":"www.gov.il","token":"qMZlz1IwqfKc6WhO6qet3OHGjLusa/tBvtVdRBurehytCsrtnVhcI81jhg/2pDc39ChRLpK7C+mzq4rg5GjqdHfTQJANTYMPAmVWrfMwL4O2AMtdLCq+QEt/inww791B+vWYq7Z0PiAq8lUPSBuB1//YGsM/BvB9daJoQwguhya+2NxIm2JK2gfMp+RnSHzGdn4LFWLuLw3lHHYzhGT8Q8+3vzRbF/vSl79P7qNAoLxGPApgGa80OA==","user_id":178592,"isAdmin":false,"expires":"2023-08-15T21:20:19.0844919+03:00"}',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'Content-Type': 'application/json; charset=UTF-8',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Referer': 'https://www.gov.il/',
        'sec-ch-ua-platform': '"Windows"',
    }

    json_data = {
        'type': 0,
        'address': address,
    }

    response = requests.post('https://ags.govmap.gov.il/Api/Controllers/GovmapApi/SearchAndLocate', headers=headers, json=json_data)
    if response.status_code == 200:
        data = response.json()
        if data['status'] != 0:
            return False
        if 'data' in data and len(data['data']) > 0 and 'Values' in data['data'][0]:
            print("data")
            return [int(value) for value in data['data'][0]['Values']]
    return False


In [59]:
def add_gush_helka_tat(df, city='תל אביב'):
    df.loc[:, 'Gush'] = np.nan
    df.loc[:, 'Helka'] = np.nan
    df.loc[:, 'Tat'] = np.nan

    df1_gov = pd.read_csv("Data/Real_Estate_TLV_GOVMAPS_1.csv", index_col=0)
    df2_gov = pd.read_csv("Data/Real_Estate_TLV_GOVMAPS_2.csv", index_col=0)
    df_gov = pd.merge(df1_gov, df2_gov, how='outer')
    df_gov[['Gush', 'Helka', 'Tat']] = df_gov['GUSHHELKATAT'].str.split('-|/', n=2, expand=True).astype(np.int32)
    df_gov['Home_number'] = df_gov['ADDRESS'].str.extract('(\d+)').astype(np.int32)
    
    # Level 0: Government Maps API
#     for index, row in df.iterrows():
#         if pd.notna(row['Home_number']):
#             home_number = int(row['Home_number'])
#             street = row['Street']
            
#             for home_n in range(0,10,2):
#                 home_number = home_number + home_n
#                 address = str(street)+' '+str(home_number)+' '+city
#                 gush_chelka = get_gush_chelka_api(address)
#                 if gush_chelka:
#                     df.loc[index, 'Gush'] = gush_chelka[0]
#                     df.loc[index, 'Helka'] = gush_chelka[1]
#                     break
            
    # Level 1: Government Maps
    for index, row in df.iterrows():
        if pd.notna(row['Home_number']) and pd.isna(row['Gush']):
            home_number = row['Home_number']
            street = row['Street']

            matches = df_gov.loc[
                (df_gov['STREENNAME'].str.contains(street)) & (df_gov['Home_number'] == home_number), ['Gush', 'Helka', 'Tat']]

            if not matches.empty:
                df.loc[index, 'Gush'] = matches['Gush'].values[0]
                df.loc[index, 'Helka'] = matches['Helka'].values[0]
                df.loc[index, 'Tat'] = matches['Tat'].values[0]

    # Level 2: Nadlan Database
    print(df['Gush'].notna().sum())
    df_nadlan = pd.read_csv("Data/Nadlan.csv")
    df_nadlan[['Gush', 'Helka', 'Tat']] = df_nadlan['GUSH'].str.split('-|/', n=2, expand=True).astype(np.int32)
    df_nadlan = df_nadlan.drop(columns='GUSH', axis=1)
    df_nadlan[['Street', 'Home_number']] = df_nadlan['DISPLAYADRESS'].str.extract('(.+)\s+(\d+)')
    df_nadlan.dropna(subset=['Home_number'], inplace=True)
    df_nadlan['Home_number'] = df_nadlan['Home_number'].astype(np.int32)

    for index, row in df.iterrows():
        if pd.notna(row['Home_number']) and pd.isna(row['Gush']):
            home_number = row['Home_number']
            street = row['Street']

            matches = df_nadlan.loc[
                (df_nadlan['Street'].str.contains(street)) & (df_nadlan['Home_number'] == home_number), ['Gush', 'Helka', 'Tat']]

            if not matches.empty:
                df.loc[index, 'Gush'] = matches['Gush'].values[0]
                df.loc[index, 'Helka'] = matches['Helka'].values[0]
                df.loc[index, 'Tat'] = matches['Tat'].values[0]

    # Level 3: Addresses
    print(df['Gush'].notna().sum())
    df_address = pd.read_csv("Data/Addresses.csv")
    for index, row in df.iterrows():
        if pd.notna(row['Home_number']):
            home_number = row['Home_number']
            street = row['Street']

            gush = df_address.loc[(df_address['ms_bayit'] == home_number) & (df_address['t_rechov'].str.contains(street)), 'ms_gush']
            helka = df_address.loc[(df_address['ms_bayit'] == home_number) & (df_address['t_rechov'].str.contains(street)), 'ms_chelka']

            if not gush.empty:
                df.at[index, 'Gush'] = gush.iloc[0]
            if not helka.empty:
                df.at[index, 'Helka'] = helka.iloc[0]

            if gush.empty and helka.empty:
                street_parts = street.split(' ')
                last_part = street_parts[-1]
                matching_streets = df_address[df_address['t_rechov'].str.contains(last_part)]
                if not matching_streets.empty:
                    gush = matching_streets.loc[matching_streets['ms_bayit'] == home_number, 'ms_gush']
                    helka = matching_streets.loc[matching_streets['ms_bayit'] == home_number, 'ms_chelka']
                    if not gush.empty:
                        df.at[index, 'Gush'] = gush.iloc[0]
                    if not helka.empty:
                        df.at[index, 'Helka'] = helka.iloc[0]

    df = df.dropna(subset=['Gush', 'Helka'])
    df.loc[:, 'Gush'] = df['Gush'].astype(np.int32)
    df.loc[:, 'Helka'] = df['Helka'].astype(np.int32)

    print(df['Gush'].notna().sum())
    return df

df = add_gush_helka_tat(df)
print(f'shape {df.shape}')


178
190
316
shape (316, 21)


C:\Users\yoavl\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [60]:
def create_building_year_dict():
    df = pd.read_csv("Data/Nadlan_clean.csv",index_col=0)
    df_sub = df.dropna(subset=['Floors','Street','Build_year'])
    max_build_years = df_sub.groupby(['Street', 'Home_number'])['Build_year'].max()
    return max_build_years.to_dict()

def fix_build_year(df):
    df = df.dropna(subset=['Street'])
    if 'Build_year' not in df.columns:
        df['Build_year'] = np.nan
        
    print(df[df['Build_year'].isna()].shape)

    # Use Nadlan data to fill missing Build_years
    nadlan_dict = create_building_year_dict()
    df.set_index(['Street', 'Home_number'], inplace=True)
    df['Build_year'].fillna(nadlan_dict, inplace=True)
    df.reset_index(inplace=True)
    
    print(df[df['Build_year'].isna()].shape)
    
    # For the ones still missing, use the average build year by street
    street_buildings_avg = df.groupby('Street')['Build_year'].transform('mean')
    df['Build_year'].fillna(street_buildings_avg, inplace=True)

    print(df[df['Build_year'].isna()].shape)
    df['Build_year'].fillna(df['Build_year'].mean(), inplace=True)
    df['Build_year'] = df['Build_year'].astype(np.int32)
    return df

df = fix_build_year(df)

(229, 21)
(118, 21)
(80, 21)


In [61]:

def fix_floors(df):
    # Loading nadlan data
    nadlan = pd.read_csv("Data/Nadlan_clean.csv", index_col=0)
    nadlan = nadlan.dropna(subset=['Floors', 'Street', 'Home_number']).reset_index(drop=True)
    df['Floors'] = np.nan

    max_floors = nadlan.groupby(['Street', 'Home_number'])['Floors'].max().reset_index()
    max_floors_dict = {(row['Street'], row['Home_number']): row['Floors'] for _, row in max_floors.iterrows()}
    
    for idx, row in df.iterrows():
        if pd.isna(row['Floors']):
            match = max_floors_dict.get((row['Street'], row['Home_number']), np.nan)
            if not pd.isna(match):
                df.at[idx, 'Floors'] = match
    
    street_avg_floors = nadlan.groupby('Street')['Floors'].mean().round().reset_index()
    street_avg_floors_dict = {row['Street']: row['Floors'] for _, row in street_avg_floors.iterrows()}
    
    for idx, row in df[df['Floors'].isna()].iterrows():
        match = street_avg_floors_dict.get(row['Street'], np.nan)
        if not pd.isna(match):
            df.at[idx, 'Floors'] = match   
            
    df.loc[df['Floor'] > df['Floors'], 'Floors'] = df['Floor']
    df['Floors'] = df['Floors'].fillna(df['Floor'])
    return df


def fix_condition(df):
    df['Condition'] = df['Condition'].apply(lambda x: 1 if x == 'new' else 0)
    df['Condition'] =  df['Condition'].astype(np.int32)
    df.rename(columns = {'Condition':'New'}, inplace = True)

    return df

df = fix_floors(df)
df = fix_condition(df)

In [62]:
def convert_coordinates(lat_long_list):
    '''
    This function use to convert UTM coordinates to ITM coordinates using outside website with selenium 
    '''
    service = Service("C:/Users/yoavl/NextRoof/chromedriver_win32/chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    driver.get("https://zvikabenhaim.appspot.com/software/ITM/")

    # locate all the input fields and button
    lat_input = driver.find_element(By.XPATH, '//*[@id="lat"]')
    lon_input = driver.find_element(By.XPATH, '//*[@id="long"]')
    convert_button = driver.find_element(By.XPATH, '//form[1]/table/tbody/tr[3]/td/input')
    east_span = driver.find_element(By.XPATH, '//*[@id="itm_east"]')
    north_span = driver.find_element(By.XPATH, '//*[@id="itm_north"]')
    
    
    coverted_list = []
    for cords in lat_long_list:
        east = cords[0]
        north = cords[1]
        if cords[0] != 0.0 and cords[1] != 0.0:
            lat_input.clear()
            lon_input.clear()

            lat_input.send_keys(cords[0])
            lon_input.send_keys(cords[1])

            convert_button.click()


            east = east_span.text
            north = north_span.text
            
            
        coverted_list.append((east,north))
        

    driver.quit()
    return coverted_list

def convert_lat_long(df):

    def get_long_lat_tuples(df):
        long_lat_tuples = []
        for index, row in df.iterrows():
            lat = row['Lat']
            long = row['Long']

            try:
                lat = float(lat)
            except:
                lat = 0.0
            try:
                long = float(long)
            except:
                long = 0.0  
            long_lat_tuples.append((lat, long))
        return long_lat_tuples
    
    new_cord = convert_coordinates(get_long_lat_tuples(df))
    
    for index, row in df.iterrows():  
        try:
            df.at[index,'Lat'] = new_cord[index][0]
            df.at[index,'Long'] = new_cord[index][1]
        except:
            pass
    return df

In [63]:
def find_avg_cords_by_street():
    df = pd.read_csv("Data/Nadlan_clean.csv",index_col=0)
    df = df.dropna(subset=['Long','Lat','Street']).reset_index(drop=True)
    df['Long'] = df['Long'].astype(int)
    df['Lat'] = df['Lat'].astype(int)
    
    # create unique streets list
    streets = df['Street'].unique()
    
    street_cords_avg = {}
    for street in streets:
        street_df = df[df['Street'] == street]
        
        lat_mean = street_df['Lat'].mean()
        long_mean = street_df['Long'].mean()

        street_cords_avg[street] = (long_mean ,lat_mean)
        
    return street_cords_avg
        
def complete_long_lat_columns(df,street_cords_avg):
    for index, row in df.iterrows():
        long = row['Long']
        lat = row['Lat']
        street = row['Street']
        
        try:
            cords = street_cords_avg[street]         
            if long == 0.0 or lat == 0.0:
                df.at[index, 'Long'] = cords[1]
                df.at[index, 'Lat'] = cords[0]      
        except:
            pass

    
    df.rename(columns = {'Lat':'Temp'}, inplace = True)
    df.rename(columns = {'Long':'Lat'}, inplace = True)
    df.rename(columns = {'Temp':'Long'}, inplace = True)

    df = df.loc[(df['Lat'] != 0.0) & (df['Long'] != 0.0)]
    
    df.loc[:, 'Long'] = df['Long'].astype(np.int32)
    df.loc[:, 'Lat'] = df['Lat'].astype(np.int32)
    
    return df
            
# df = convert_lat_long(df)
# df =complete_long_lat_columns(df,find_avg_cords_by_street())
print(df.shape)

(316, 22)


In [64]:
def add_year(df):
    current_year = datetime.now().year
    df['Year'] = current_year
    df.assign(profit='NAN')
    df['Year'].astype(np.int32)
    return df


In [65]:
def update_neighborhood_street_token(df):
    df['Street'] = df['Street'].str.strip().str.replace("'", "")
    df['Neighborhood'] = df['Neighborhood'].str.strip()

    replacements_street = {
            "כרמייה": "כרמיה"
        }

    replacements_neighborhood = {
            "'": "",
            "-": " ",
            "נווה אביבים": "נווה אביבים וסביבתה",
            "לב תל אביב, לב העיר צפון": "הצפון הישן החלק הצפוני",
            "נוה": "נווה",
            "גני צהלה, רמות צהלה": "גני צהלה",
            "נווה אליעזר וכפר שלם מזרח": "כפר שלם מזרח נווה אליעזר",
            "גני שרונה, קרית הממשלה": "גני שרונה",
            "הצפון הישן   דרום": "הצפון החדש החלק הדרומי",
            "מכללת תל אביב יפו, דקר": "דקר"
        }

    for old, new in replacements_street.items():
        df['Street'] = df['Street'].str.replace(old, new)

    for old, new in replacements_neighborhood.items():
        df['Neighborhood'] = df['Neighborhood'].str.replace(old, new)
    return df 


def street_and_neighborhood_rank(df, column):
    df = update_neighborhood_street_token(df)
    df = df.dropna(subset=['Neighborhood','Street']).reset_index(drop=True)
    df_nadlan = pd.read_csv('Data/Nadlan_clean.csv', index_col=0)

        
    rank = df_nadlan.groupby(['Year', column])\
        .apply(lambda x: np.nan if x['Size'].sum() == 0 else x['Price'].sum() / x['Size'].sum())\
        .unstack().to_dict(orient="index")

    new_column_name = column + '_rank'
    df[new_column_name] = np.nan

    for index, row in df.iterrows():
        year = row['Year']
        item = row[column]
        found_street = False

        for nadlan_street in df_nadlan[column].unique():
            if item == nadlan_street:
                found_street = True
                item = nadlan_street
                break

        if not found_street and  found_street != False:
            for nadlan_street in df_nadlan[column].unique():
                if item in nadlan_street:
                    item = nadlan_street
                    found_street = True
                    break
                    
                check_parts = nadlan_street.split(' ')
                print(check_parts)
                if item in check_parts:
                    item = nadlan_street
                    found_street = True
                    break

        if found_street:
            for potential_year in [year, year-1, year-2 , year-3]:
                try:
                    df.at[index, new_column_name] = rank[potential_year][item]
                    break
                except KeyError:
                    continue               
        else:
            pass
            
    mean = df[new_column_name].mean()
    df[new_column_name] = df[new_column_name].fillna(mean)
    df[new_column_name] = df[new_column_name].astype(np.int32)
    return df



df = add_year(df)
df = street_and_neighborhood_rank(df,'Neighborhood')
df = street_and_neighborhood_rank(df,'Street')
df = street_and_neighborhood_rank(df,'Gush')

,Street,Home_number,Item_id,Lat,Long,City,Rooms,Neighborhood,Floor,Baths,...,Asset_type,Images,Gush,Helka,Tat,Floors,Year,Neighborhood_rank,Street_rank,Gush_rank
0,עין הקורא,18,dDWKk7Qtpnf,32.057245,34.782693,תל אביב יפו,2.0,נווה שאנן,5.0,NaN,...,flat,"['/bulletin/dDWKk7Qtpnf/bosRcd.jpg', '/bulleti...",6944,82,NaN,5.0,2023,51672,48305,48305
1,נחום,38,btbIetNpaGY,32.089581,34.777868,תל אביב יפו,3.5,הצפון הישן החלק הצפוני,1.0,NaN,...,flat,"['/bulletin/btbIetNpaGY/kB7gn.jpg', '/bulletin...",6132,289,NaN,3.0,2023,60675,54831,51200
2,בזל,14,bgWP3MofHTr,32.090116,34.777054,תל אביב יפו,3.0,הצפון הישן החלק הצפוני,7.0,NaN,...,flat,"['/bulletin/bgWP3MofHTr/32S0R.jpg', '/bulletin...",6956,118,NaN,7.0,2023,60675,54831,51200
3,רמה,3,bxhbc66ZpTP,32.119466,34.841451,תל אביב יפו,4.0,נווה שרת,6.0,NaN,...,flat,['bulletins/b5d2238f-1fa1-43d5-9b61-07166c2e50...,7065,142,72.0,24.0,2023,38921,42523,41314
4,בני אפרים,209,doXEXSjV8U8,32.111256,34.816224,תל אביב יפו,4.0,מעוז אביב א,1.0,NaN,...,flat,"['/bulletin/doXEXSjV8U8/mLW6R.jpg', '/bulletin...",6636,654,4.0,5.0,2023,51484,30208,41947


In [66]:

def check_for_match(df_nadlan, parcel):
    parcel = int(parcel)
    parcel_offsets = [parcel - 1, parcel + 1, parcel + 2, parcel - 2, parcel + 3, parcel - 3, parcel + 4, parcel - 4]
    for parcel_offset in parcel_offsets:
        match = df_nadlan.loc[(df_nadlan['Gush_Helka'] == str(parcel_offset)), 'Helka_rank']
        if not match.empty:
            return match
    return match

def parcel_rank(df):
    count = 0
    df = df.dropna(subset=['Gush','Helka']).reset_index(drop=True)
    df_nadlan = pd.read_csv('Data/Nadlan_clean.csv', index_col=0)
    df_nadlan['Gush_Helka'] = df_nadlan['Gush'].astype(str) + df_nadlan['Helka'].astype(str)
    
    column = 'Gush_Helka'
    df[column] = df['Gush'].astype(str) + df['Helka'].astype(str)
    df[column] = df[column].str.replace('.', '', regex=False)
    
    for index, row in df.iterrows():
        gush_helka = row['Gush_Helka']
        match = df_nadlan.loc[(df_nadlan['Gush_Helka'] == gush_helka),'Helka_rank']
        print(match)
        if match.empty:
            match = check_for_match(df_nadlan, gush_helka)
    
        if not match.empty:
            df.at[index, 'Helka_rank'] = match.values[0]
        
        if match.empty:
            count+=1
            
    df['Helka_rank'] = df['Helka_rank'].fillna(df['Helka_rank'].mean()).astype(np.int32)
    print(count)
    return df


df = df.dropna(subset = ['Size'])
df['Year'] = df['Year'].astype(np.int32)
df['Size'] = df['Size'].astype(np.int32)
df = parcel_rank(df)

Series([], Name: Helka_rank, dtype: int64)
Series([], Name: Helka_rank, dtype: int64)
Series([], Name: Helka_rank, dtype: int64)
41313    29544
50080    29544
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
1125     36413
1843     36413
4085     36413
5439     36413
9186     36413
11040    36413
11551    36413
11867    36413
12580    36413
18195    36413
21355    36413
21614    36413
27378    36413
39110    36413
39796    36413
41274    36413
44916    36413
47805    36413
48564    36413
49055    36413
50422    36413
57725    36413
59686    36413
60276    36413
62072    36413
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
Series([], Name: Helka_rank, dtype: int64)
Series([], Name: Helka_rank, dtype: int64)
41313    29544
50080    29544
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
1125     36413
1843     36413
4085     36413
5439     36413
9186     36413
11040    36413
11551    36413
11867    36413
12580   

Series([], Name: Helka_rank, dtype: int64)
13682    73226
51491    73226
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
Series([], Name: Helka_rank, dtype: int64)
3414     28272
3491     28272
4495     28272
4633     28272
6197     28272
8899     28272
10286    28272
16529    28272
19755    28272
20405    28272
21273    28272
21276    28272
43574    28272
52007    28272
55374    28272
55495    28272
58454    28272
61535    28272
Name: Helka_rank, dtype: int64
1021     33691
4514     33691
5675     33691
10910    33691
11446    33691
11855    33691
12184    33691
12423    33691
12869    33691
13093    33691
15784    33691
34537    33691
42101    33691
43236    33691
47640    33691
59590    33691
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
2937    68607
Name: Helka_rank, dtype: int64
782      39030
1122     39030
1461     39030
1763     39030
4447     39030
6510     39030
9646     39030
13087    39030
13890    39030
14491    39030


9284     43812
24775    43812
37492    43812
Name: Helka_rank, dtype: int64
2453     45706
6907     45706
23900    45706
27499    45706
41903    45706
47267    45706
48278    45706
60742    45706
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
5774     49017
16072    49017
27725    49017
33382    49017
44476    49017
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
625      57434
1231     57434
2323     57434
4048     57434
7827     57434
8368     57434
8944     57434
10752    57434
14201    57434
53693    57434
Name: Helka_rank, dtype: int64
5066     58091
11566    58091
16477    58091
19876    58091
25057    58091
28670    58091
30747    58091
32112    58091
34780    58091
36087    58091
36864    58091
44941    58091
48133    58091
Name: Helka_rank, dtype: int64
13458    66774
Name: Helka_rank, dtype: int64
2453     45706
6907     45706
23900    45706
27499    45706
41903    45706
47267    45706
48278    45706
60742    45706
Name: He

Series([], Name: Helka_rank, dtype: int64)
10744    36624
Name: Helka_rank, dtype: int64
41313    29544
50080    29544
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
1132     48927
3458     48927
4611     48927
6092     48927
6205     48927
10263    48927
13358    48927
14356    48927
14540    48927
14575    48927
14759    48927
16191    48927
16447    48927
17293    48927
17358    48927
18028    48927
19065    48927
23938    48927
25321    48927
26104    48927
26337    48927
28570    48927
30599    48927
31502    48927
35646    48927
36284    48927
36483    48927
37757    48927
40646    48927
43430    48927
47446    48927
49175    48927
49265    48927
49389    48927
49605    48927
51842    48927
51940    48927
57721    48927
58025    48927
59647    48927
61587    48927
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
782      39030
1122     39030
1461     39030
1763     39030
4447     39030
6510     39030
9646     39030
13087    3903

1222     53334
1979     53334
4809     53334
11059    53334
13151    53334
20306    53334
59709    53334
Name: Helka_rank, dtype: int64
1153     56414
1715     56414
57731    56414
59780    56414
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
Series([], Name: Helka_rank, dtype: int64)
5456     35525
9281     35525
12339    35525
34839    35525
40489    35525
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
Series([], Name: Helka_rank, dtype: int64)
Series([], Name: Helka_rank, dtype: int64)
28520    55415
34636    55415
38385    55415
Name: Helka_rank, dtype: int64
Series([], Name: Helka_rank, dtype: int64)
31307    107177
Name: Helka_rank, dtype: int64
873      56901
12576    56901
20313    56901
25342    56901
40727    56901
45477    56901
46322    56901
59499    56901
Name: Helka_rank, dtype: int64
5259     85984
13165    85984
15037    85984
19524    85984
20356    85984
29357    85984
29782    85984
30360    85984
Name: Helka_ran

,Street,Home_number,Item_id,Lat,Long,City,Rooms,Neighborhood,Floor,Baths,...,Gush,Helka,Tat,Floors,Year,Neighborhood_rank,Street_rank,Gush_rank,Gush_Helka,Helka_rank
0,עין הקורא,18,dDWKk7Qtpnf,32.057245,34.782693,תל אביב יפו,2.0,נווה שאנן,5.0,NaN,...,6944,82,NaN,5.0,2023,51672,48305,48305,694482,43160
1,נחום,38,btbIetNpaGY,32.089581,34.777868,תל אביב יפו,3.5,הצפון הישן החלק הצפוני,1.0,NaN,...,6132,289,NaN,3.0,2023,60675,54831,51200,6132289,49222
2,בזל,14,bgWP3MofHTr,32.090116,34.777054,תל אביב יפו,3.0,הצפון הישן החלק הצפוני,7.0,NaN,...,6956,118,NaN,7.0,2023,60675,54831,51200,6956118,50239
3,רמה,3,bxhbc66ZpTP,32.119466,34.841451,תל אביב יפו,4.0,נווה שרת,6.0,NaN,...,7065,142,72.0,24.0,2023,38921,42523,41314,7065142,29544
4,בני אפרים,209,doXEXSjV8U8,32.111256,34.816224,תל אביב יפו,4.0,מעוז אביב א,1.0,NaN,...,6636,654,4.0,5.0,2023,51484,30208,41947,6636654,49222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,מיטב,5,jr57aT9WtiR,32.065739,34.794772,תל אביב יפו,3.5,ביצרון,1.0,NaN,...,7096,64,NaN,27.0,2023,42463,42658,42385,709664,40741
309,"רש""י",6,dSgqu5sF5xt,32.072170,34.771831,תל אביב יפו,4.0,לב תל אביב החלק המערבי,1.0,NaN,...,7226,69,1.0,10.0,2023,51484,63592,64782,722669,57305
310,חובבי ציון,38,c7gp7MIUGIH,32.077456,34.771069,תל אביב יפו,2.0,הצפון הישן החלק הדרום מערבי,0.0,NaN,...,7091,148,8.0,4.0,2023,51484,78906,72222,7091148,61410
311,וינגייט,1,3GvEl1261Zk,32.060150,34.787733,תל אביב יפו,2.0,יד אליהו,31.0,NaN,...,7108,51,NaN,31.0,2023,40312,54831,48574,710851,45349


In [68]:
df.to_csv('Data/madlan_data_clean.csv', index=False)
print(df.shape)